# Metadata

**L1 Taxonomy** - Backend Development

**L2 Taxonomy** - Caching Strategies

**Subtopic** - Integrating an in-memory TTL cache to expire old entries

**Use Case** - Develop a Python script that implements an in-memory TTL cache using the 'cachetools' library. The script should allow for storing key-value pairs in the cache with a specified TTL (Time To Live) value. After the TTL value has expired, the key-value pair should be automatically removed from the cache.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
cachetools==6.1.0
```


# Prompt
Problem Statement:
- Develop a Python script that implements an in-memory TTL cache using the 'cachetools' library.
- The script should allow for storing key-value pairs in the cache with a specified TTL (Time To Live) value.
- After the TTL value has expired, the key-value pair should be automatically removed from the cache.

Input Format:
- A command string: either `SET`, `GET`, `SLEEP`, or `EXIT`.
- For `SET`, provide:
  - A string key.
  - A string value.
  - An integer TTL in seconds.
- For `GET`, provide:
  - A string key.
- For `SLEEP`, provide:
  - An integer number of seconds to simulate delay.
- For `EXIT`, no additional input is required.

Input Constraints:
- The key must be a non-empty alphanumeric string of at most 64 characters.
- The value must be a string of at most 256 characters.
- TTL (Time To Live) must be an integer in the range [1, 86400] (i.e., between 1 second and 24 hours).
- Commands are case-sensitive and must be one of: SET, GET, SLEEP, EXIT.
- Maximum number of total commands: 10^4.

Output Format:
- For each `GET <key>` command:
  - If the key exists and has not expired, print the corresponding value.
  - If the key does not exist or has expired, print `NULL`.
- For other commands (`SET`, `SLEEP`, `EXIT`), no output is produced.

Example:
```python
Input:
SET session123 Alice 3
GET session123
SLEEP 4
GET session123
EXIT

Output:
Alice
NULL
```

Function Signature:
```python
def ttl_cache_simulator(commands: list[str]) -> list[str]:
```

# Requirements
Explicit Requirements:
- Use the `cachetools` library to implement an in-memory TTL cache.
- Support storing key-value pairs with a TTL (Time To Live).
- Expired keys should be automatically removed or become inaccessible.
- Support the following commands: SET, GET, SLEEP, EXIT.
- Implement input/output processing via a function: ttl_cache_simulator(commands: list[str]) -> list[str].
- Commands after EXIT should be ignored, and the EXIT command signals termination.
- All commands must appear in UPPERCASE exactly as specified (SET, GET, etc.).
- Assume an arbitrary large maxsize (e.g., 10^4) so that eviction due to size does not occur during simulation.

Implicit Requirements:
- The cache must behave correctly under simulated delays (`SLEEP`) by advancing a virtual clock (no real sleep).
- Do not print anything from the function; collect outputs in a list and return it.
- TTL countdown starts at the time of the SET command, relative to the simulated time.
- GET should return 'NULL' if the key is missing or has expired.
- The function must handle up to 10^4 commands efficiently.
- All keys and values are treated as strings; TTL is an integer.

Solution Expectations:
- Correctly parse and execute all supported commands: SET, GET, SLEEP, EXIT.
- Maintain an in-memory TTL cache using the `cachetools.TTLCache` class.
- Simulate time using a virtual clock (e.g., an integer counter).
- Store each key-value pair with the specified TTL in seconds.
- Ensure expired keys are inaccessible after TTL has elapsed.
- For each GET command, append the corresponding output ("value" or "NULL") to a result list.
- Return the result list after processing all commands.
- Do not produce side effects like printing or reading input; the function must be deterministic.
- Efficiently handle up to 10^4 commands without timeouts or memory issues.

Edge Cases and Behavior:
- Setting a key with TTL=1:
  - The key should expire exactly after 1 second of simulated time.
  
- Accessing a key right before expiry:
  - Should still return the correct value if TTL hasn't fully elapsed.

- Accessing a key right after expiry:
  - Should return 'NULL' as the key has expired.

- Re-setting an existing key:
  - Should overwrite the value and reset its TTL countdown from the new simulated time.

- GET on a non-existent key:
  - Should return 'NULL'.

- Consecutive SLEEP commands:
  - Their durations should accumulate to affect TTL expiry correctly.

- SET with maximum allowed TTL (86400 seconds):
  - Should retain the key for up to 24 simulated hours.

- Large number of commands (10^4):
  - Must be processed efficiently with correct behavior maintained.

- SLEEP without any prior SET:
  - Should not cause errors; simulation should simply advance virtual time.

- GET after EXIT or unrelated commands:
  - Should not be processed; only commands before EXIT should be executed.

Solution Constraints:
- You must use the `cachetools` library, specifically `cachetools.TTLCache`.
- The total number of commands will not exceed 10^4.
- Keys are unique strings with a maximum length of 64 characters.
- Values are strings with a maximum length of 256 characters.
- TTL values are integers in the range [1, 86400] (1 second to 24 hours).
- The solution must run in O(1) time per command on average.
- No global variables or state should persist outside the function.
- The function must return a list of output strings; it should not print anything.

In [ ]:
# code

"""Simulate an in-memory TTL cache using cachetools with virtual time."""

import cachetools
from typing import List


def ttl_cache_simulator(commands: List[str]) -> List[str]:
    """Simulate a TTL cache using cachetools.TTLCache with virtual clock."""
    virtual_time: int = 0
    outputs: List[str] = []

    def timer_func() -> int:
        return virtual_time

    # Cache with max TTL to avoid premature eviction by the cache itself
    cache = cachetools.TTLCache(maxsize=10 ** 4, ttl=86400, timer=timer_func)
    expiry_times = {}  # key -> expiry timestamp (virtual_time + ttl)

    for command_str in commands:
        parts = command_str.split()
        command = parts[0]

        if command == "EXIT":
            break

        elif command == "SET":
            key, value, ttl_str = parts[1], parts[2], parts[3]
            ttl = int(ttl_str)
            cache[key] = value
            expiry_times[key] = virtual_time + ttl  # track expiry

        elif command == "GET":
            key = parts[1]
            expiry = expiry_times.get(key)
            value = cache.get(key)

            if value is None or expiry is None or expiry <= virtual_time:
                # expired or missing
                cache.pop(key, None)
                expiry_times.pop(key, None)
                outputs.append("NULL")
            else:
                outputs.append(value)

        elif command == "SLEEP":
            seconds = int(parts[1])
            virtual_time += seconds

    return outputs


In [ ]:
# tests

"""
Unit tests for ttl_cache_simulator using cachetools library,
ensuring correct per-key TTL logic and simulated time advancement.
"""

import unittest
from main import ttl_cache_simulator


class TestTTLCacheSimulator(unittest.TestCase):
    """Test suite for the ttl_cache_simulator function."""

    def test_basic_set_and_get(self):
        """Test simple SET and GET commands."""
        cmds = ["SET k1 v1 10", "GET k1", "EXIT"]
        self.assertEqual(ttl_cache_simulator(cmds), ["v1"])

    def test_get_nonexistent_key_returns_null(self):
        """GET for a key never SET returns NULL."""
        cmds = ["GET keyX", "EXIT"]
        self.assertEqual(ttl_cache_simulator(cmds), ["NULL"])

    def test_expiry_after_ttl(self):
        """Key expires after SLEEP equals or exceeds TTL."""
        cmds = [
            "SET sid A 2", "SLEEP 2",
            "GET sid", "EXIT"
        ]
        self.assertEqual(ttl_cache_simulator(cmds), ["NULL"])

    def test_access_just_before_and_after_expiry(self):
        """GET before expiry returns value, after returns NULL."""
        cmds = [
            "SET x val 5", "SLEEP 4", "GET x", "SLEEP 1", "GET x", "EXIT"
        ]
        self.assertEqual(ttl_cache_simulator(cmds), ["val", "NULL"])

    def test_multiple_keys_different_ttls(self):
        """Multiple keys with different TTLs handled independently."""
        cmds = [
            "SET a va 3", "SET b vb 7", "SLEEP 4", "GET a", "GET b", "EXIT"
        ]
        self.assertEqual(ttl_cache_simulator(cmds), ["NULL", "vb"])

    def test_overwrite_key_resets_ttl(self):
        """Reseting a key via SET updates its expiry."""
        cmds = [
            "SET k v1 2", "SLEEP 1", "SET k v2 4", "SLEEP 2", "GET k", "EXIT"
        ]
        self.assertEqual(ttl_cache_simulator(cmds), ["v2"])

    def test_key_min_ttl(self):
        """Key with TTL=1 expires after one tick."""
        cmds = [
            "SET k1 v 1", "GET k1", "SLEEP 1", "GET k1", "EXIT"
        ]
        self.assertEqual(ttl_cache_simulator(cmds), ["v", "NULL"])

    def test_key_max_ttl(self):
        """Key with max TTL remains valid up to the 24 hour mark."""
        cmds = [
            "SET k1 longval 86400", "SLEEP 86399", "GET k1", "SLEEP 1", "GET k1", "EXIT"
        ]
        self.assertEqual(ttl_cache_simulator(cmds), ["longval", "NULL"])

    def test_multiple_gets(self):
        """GET multiple times before expiry yields value."""
        cmds = [
            "SET a 1 10", "GET a", "GET a", "SLEEP 9", "GET a", "EXIT"
        ]
        self.assertEqual(ttl_cache_simulator(cmds), ["1", "1", "1"])

    def test_exit_terminates_processing(self):
        """No commands processed after EXIT."""
        cmds = [
            "SET k1 v1 5",
            "EXIT",
            "GET k1",
            "SLEEP 100",
            "GET k1"
        ]
        self.assertEqual(ttl_cache_simulator(cmds), [])

    def test_zero_length_input(self):
        """Empty input yields empty output."""
        self.assertEqual(ttl_cache_simulator([]), [])

    def test_sleep_no_set(self):
        """SLEEP before SET is allowed and has no effect."""
        cmds = [
            "SLEEP 10", "SET k foo 3", "SLEEP 2", "GET k", "SLEEP 1", "GET k", "EXIT"
        ]
        self.assertEqual(ttl_cache_simulator(cmds), ["foo", "NULL"])

    def test_overlapping_set_calls(self):
        """Overlapping SETs for same key reset TTL from now."""
        cmds = [
            "SET x v1 2", "SLEEP 1", "SET x v2 4", "SLEEP 2", "GET x", "EXIT"
        ]
        self.assertEqual(ttl_cache_simulator(cmds), ["v2"])

    def test_batch_many_keys(self):
        """Handles many keys and their expiry distinctly."""
        cmds = [
            *(f"SET k{i} v{i} {i+1}" for i in range(20)),
            "SLEEP 10",
            *(f"GET k{i}" for i in range(20)),
            "EXIT"
        ]
        expected = ["NULL" if i < 10 else f"v{i}" for i in range(20)]
        self.assertEqual(ttl_cache_simulator(list(cmds)), expected)

    def test_max_key_value_length(self):
        """Max-length keys and values stored and expire properly."""
        k = "K" * 64
        v = "V" * 256
        cmds = [f"SET {k} {v} 3", "GET " + k, "SLEEP 3", "GET " + k, "EXIT"]
        self.assertEqual(ttl_cache_simulator(cmds), [v, "NULL"])

    def test_many_gets_after_expiry(self):
        """Multiple consecutive GETs after expiry all return NULL."""
        cmds = [
            "SET temp v 2", "SLEEP 2",
            "GET temp", "GET temp", "EXIT"
        ]
        self.assertEqual(ttl_cache_simulator(cmds), ["NULL", "NULL"])

    def test_sleep_zero_is_effective(self):
        """SLEEP 0 does not advance virtual time."""
        cmds = [
            "SET a va 2", "SLEEP 0", "GET a", "SLEEP 2", "GET a", "EXIT"
        ]
        self.assertEqual(ttl_cache_simulator(cmds), ["va", "NULL"])

    def test_interleaved_set_sleep_get(self):
        """Test a mix of interleaved operations."""
        cmds = [
            "SET a 1 3", "SLEEP 1",
            "SET b 2 4", "SLEEP 2",
            "GET a", "GET b",   # a expired, b remains
            "SLEEP 2",
            "GET b", "EXIT"
        ]
        self.assertEqual(ttl_cache_simulator(cmds), ["NULL", "2", "NULL"])


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/f62402e0-483b-4947-859a-7fbabaa8f7c4?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F83894%2Fview

#### Model code:

```python
# code generated by the model
from cachetools import TTLCache
import time

def ttl_cache_simulator(commands):
    # Initialize a cache with a very large size limit, so we only deal with TTL evictions
    # Issue: Misuse of TTLCache. Initializing with ttl=0 effectively disables TTLCache's built-in time-to-live functionality, making it behave like a basic dictionary. The problem requires using TTLCache for its TTL capabilities.
    cache = TTLCache(maxsize=10**4, ttl=0)  # Initialize with ttl=0 since we'll simulate TTL ourselves
    
    # Initialize a dictionary to keep track of exact expiration times
    # Issue: Redundant data storage and manual TTL management. This dictionary is necessary because TTLCache's core TTL feature is not being utilized for per-item TTLs, and the virtual clock is not integrated with TTLCache's internal timer.
    expiration_times = {}
    
    # Simulated current time (in seconds)
    current_time = 0
    
    # List to capture the output
    result = []

    for command in commands:
        parts = command.split()
        op = parts[0]
        
        if op == 'SET':
            key = parts[1]
            value = parts[2]
            ttl = int(parts[3])
            
            # Calculate the absolute expiration time
            expiration_time = current_time + ttl
            expiration_times[key] = expiration_time
            
            # Set the entry in the cache with current time as the TTL
            # Using the difference between current_time and expiration_time to simulate
            # Issue: Misuse of TTLCache. Here, the code is storing a tuple `(value, expiration_time)` and then manually checking this `expiration_time` later. TTLCache is meant to store just the `value` and manage its own expiry based on its internal timer and the `ttl` provided at initialization or per-item if directly supported (which it isn't directly for `cache[key] = value`). The `TTLCache` instance itself is not aware of the per-item `ttl` from the command.
            cache[key] = (value, expiration_time)
            cache[key] = (value, expiration_time)

        elif op == 'GET':
            key = parts[1]
            expiration_time = expiration_times.get(key)
            
            # Retrieve the value and its expiration time
            if key in cache:
                value, exp_time = cache[key]
                # Issue: Manual TTL check. This is what TTLCache should be doing automatically if correctly configured with a virtual timer.
                if current_time < exp_time:
                    result.append(value)
                else:
                    result.append('NULL')
                    # Remove expired key manually
                    # Issue: Manual removal. TTLCache should automatically handle removal (or inaccessibility) when its internal timer indicates expiry.
                    cache.pop(key, None)
                    expiration_times.pop(key, None)
            else:
                result.append('NULL')

        elif op == 'SLEEP':
            sleep_duration = int(parts[1])
            current_time += sleep_duration

            # Remove expired keys manually from cache
            # Issue: Inefficient manual expiry. Iterating over all keys (O(N) operation) to remove expired ones contradicts the O(1) average time per command constraint, especially for 10^4 commands. `TTLCache` handles expiry more efficiently, typically on access or through its internal mechanisms.
            for key in list(cache.keys()):
                if current_time >= expiration_times[key]:
                    cache.pop(key, None)
                    expiration_times.pop(key, None)

        elif op == 'EXIT':
            # Issue: Missing "Commands after EXIT should be ignored" logic for the outer loop. While `return result` correctly exits the function, the prompt implies that `EXIT` is not the last command in commands` list, subsequent commands in the list should be skipped, not just that the function should return.
            return result

    return result

# Example usage
commands = [
    "SET session123 Alice 3",
    "GET session123",
    "SLEEP 4",
    "GET session123",
    "EXIT"
]

output = ttl_cache_simulator(commands)
print(output)
```